In [1]:
import os

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, RandomResizedCrop, RandomRotation
from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from Module.models.timm_image_encoder import TimmImageEncoder
from Module.dataset.image_folder import ImageFolder

from mobile_sam import SamPredictor, sam_model_registry
from mobile_sam.modeling.sam import Sam

C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\timm\models\registry.py:4: FutureWarning: Import

## Config setting

In [2]:
lr = 3e-4
bs = 4
N_epoch = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cuda


## Models
### Teacher model

In [4]:
model_type_t = 'vit_l'
checkpoint_t = './Weights/sam_vit_l_0b3195.pth'

model_t = sam_model_registry[model_type_t](checkpoint=checkpoint_t)
model_t.to(device)
model_t.eval();

### Student model

In [5]:
model_type_s = 'resnet18'

model_s = TimmImageEncoder('resnet18', pretrained=True)
model_s.to(device);
model_s.train();

## Dataset
### Train dataset

### Transform

In [6]:
transform = A.Compose([
    A.ColorJitter(p=0.7),
    A.RandomResizedCrop((1024, 1024)),
    A.Rotate((-90,90)),
    A.Normalize(
        mean=[123.675/255, 116.28/255, 103.53/255],
        std=[58.395/255, 57.12/255, 57.375/255]
    ),
    ToTensorV2(),
    ])

In [7]:
dataset_dir = r"D:\WaterSegmentation\Datasets\DANU_WS_v1\train\images"

In [8]:
dataset = ImageFolder(dataset_dir, 
                      transform=transform)

In [9]:
loader = DataLoader(dataset, shuffle=True, batch_size=bs, num_workers=8)

## Train setting
### Loss

In [10]:
loss_function = F.huber_loss

#loss_function = F.mse_loss

### Optimizer

In [11]:
optimizer = torch.optim.Adam(model_s.parameters(), 
                             lr=lr)

### Save dir

In [12]:
output_checkpoint_dir = "./runs/"
output_checkpoint_dir = os.path.join(output_checkpoint_dir, "241111_vit-l_to_resnet18_v1")

if not os.path.exists(output_checkpoint_dir):
    os.makedirs(output_checkpoint_dir)
    
output_checkpoint_path = os.path.join(output_checkpoint_dir, 
                                      "Nanosam_encoder.pth")

## Train

In [13]:
for epoch in range(N_epoch):
    epoch_loss = 0.

    for sample in tqdm(iter(loader)):
        sample = sample.cuda()
        #sample_s = F.interpolate(sample, (512, 512), mode="area")

        ## Teacher model
        with torch.no_grad():
            feat_t = model_t.image_encoder(sample)

        ## Init_gradient
        optimizer.zero_grad()
        
        ## Student model
        feat_s = model_s(sample)

        loss = loss_function(feat_s, feat_t)

        ##update
        loss.backward()
        optimizer.step()
        epoch_loss += float(loss)

    epoch_loss /= len(loader)
    print(f"{epoch} Epoch -Loss: {epoch_loss}")
    
    torch.save({
        "model": model_s.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch}, output_checkpoint_path)
        
    
    plt.figure(figsize=(10, 10))
    plt.subplot(131)
    plt.title("Image")
    plt.imshow(sample[0].detach().cpu().permute(1, 2, 0))
    plt.subplot(132)
    plt.title("Teacher")
    plt.imshow(feat_t[0, 0].detach().cpu())
    plt.subplot(133)
    plt.title("Student")
    plt.imshow(feat_s[0, 0].detach().cpu())
    plt.savefig(os.path.join(output_checkpoint_dir, 
                             f"epoch_{str(epoch).zfill(3)}.png"))
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [10:01<00:00,  1.13s/it]


0 Epoch -Loss: 0.005594294662657369


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.6399999].
100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [09:56<00:00,  1.12s/it]


1 Epoch -Loss: 0.004398151685105623


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.8610326..1.8208278].
100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [09:57<00:00,  1.12s/it]


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.552854].
100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [09:56<00:00,  1.12s/it]


98 Epoch -Loss: 0.0021872937738811387


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.9295317..2.0299783].
100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [09:56<00:00,  1.12s/it]


99 Epoch -Loss: 0.002173075870404202


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.6399999].


## Visualization

In [14]:
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(feat_t[0, 0].detach().cpu())
plt.subplot(122)
plt.imshow(feat_s[0, 0].detach().cpu())
plt.savefig(os.path.join(output_checkpoint_dir, 
                         f"epoch_{epoch}.png"))
plt.close()